In [ ]:
pip install pyspark

     |████████████████████████████████| 212.3MB 72kB/s 
     |████████████████████████████████| 204kB 50.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=9de548daf86eb1698f151c5ed81e69c50a01d177ce581a2a95abb901df6af13d
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Bank project").getOrCreate()

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/bank.csv",header=True,inferSchema=True)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol("job").setOutputCol("job_index")
df1 = indexer.fit(df).transform(df)


In [ ]:
df1.toPandas()

In [ ]:
indexer.setInputCol("marital").setOutputCol("marital_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("education").setOutputCol("education_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("default").setOutputCol("default_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("housing").setOutputCol("housing_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("loan").setOutputCol("loan_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("contact").setOutputCol("contact_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("month").setOutputCol("month_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("poutcome").setOutputCol("poutcome_index")
df1 = indexer.fit(df1).transform(df1)
indexer.setInputCol("deposit").setOutputCol("label")
df1 = indexer.fit(df1).transform(df1)

In [ ]:
df1.toPandas()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,job_index,marital_index,education_index,default_index,housing_index,loan_index,contact_index,month_index,poutcome_index,deposit_index
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes,4.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no,1.0,1.0,2.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no,4.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [ ]:
feature_name=['age','balance','day','duration','campaign','pdays','previous','job_index','marital_index','education_index','default_index','housing_index','loan_index','contact_index',	'month_index'	,'poutcome_index']

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler()
assembler.setInputCols(feature_name).setOutputCol("features")
transformed_data = assembler.transform(df1)

In [ ]:
transformed_data.toPandas()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit,job_index,marital_index,education_index,default_index,housing_index,loan_index,contact_index,month_index,poutcome_index,label,features
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,"(59.0, 2343.0, 5.0, 1042.0, 1.0, -1.0, 0.0, 3...."
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,"(56.0, 45.0, 5.0, 1467.0, 1.0, -1.0, 0.0, 3.0,..."
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,"(41.0, 1270.0, 5.0, 1389.0, 1.0, -1.0, 0.0, 2...."
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes,4.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,"(55.0, 2476.0, 5.0, 579.0, 1.0, -1.0, 0.0, 4.0..."
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,"(54.0, 184.0, 5.0, 673.0, 2.0, -1.0, 0.0, 3.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,33,blue-collar,single,primary,no,1,yes,no,cellular,20,apr,257,1,-1,0,unknown,no,1.0,1.0,2.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,"[33.0, 1.0, 20.0, 257.0, 1.0, -1.0, 0.0, 1.0, ..."
11158,39,services,married,secondary,no,733,no,no,unknown,16,jun,83,4,-1,0,unknown,no,4.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,"(39.0, 733.0, 16.0, 83.0, 4.0, -1.0, 0.0, 4.0,..."
11159,32,technician,single,secondary,no,29,no,no,cellular,19,aug,156,2,-1,0,unknown,no,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,"(32.0, 29.0, 19.0, 156.0, 2.0, -1.0, 0.0, 2.0,..."
11160,43,technician,married,secondary,no,0,no,yes,cellular,8,may,9,2,172,5,failure,no,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,"(43.0, 0.0, 8.0, 9.0, 2.0, 172.0, 5.0, 2.0, 0...."


In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features',labelCol='label', maxIter=30)

In [ ]:
M = model.fit(training_data)

In [ ]:
predictions = M.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(predictions))

Logistic Regression Accuracy: 0.8019981834695731
